Import Depencies


In [11]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import numpy as np
import time
import regex
import re
from collections import Counter

# import gmaps

# # Import API key
# from api_keys import geoapify_key
# from api_keys import gmaps_key

# gmaps.configure(api_key={gmaps_key})


Set header to circumvent webscraping blocker


In [2]:
!pip install regex

     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------- --------- 30.7/41.5 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 666.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/269.2 kB ? eta -:--:--
   --------- ------------------------------ 61.4/269.2 kB 3.4 MB/s eta 0:00:01
   --------------------------------- ------ 225.3/269.2 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 269.2/269.2 kB 2.8 MB/s eta 0:00:00


In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

Create lists to store **responses**


In [3]:
# data to scrape
job_title = []
company_name = []
short_description = []
job_location = []
job_classification = []
job_subclassification = []
job_site = []

job_counter = 0

In [4]:
headers = {
     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

Create list of scraping variables (job fields, pages[4])


In [5]:
job_fields = ['Data-Analyst', 'Data-Engineer']
page_scrapes = ['', '?page=2', '?page=3','?page=4','?page=5']

Scrape data


In [6]:
for job_field in job_fields:
    for page in page_scrapes:
        url = f'https://www.seek.com.au/{job_field}-jobs/in-All-Australia{page}'
        
        # Make a request to the URL
        response = requests.get(url, headers=headers)
        
        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to retrieve data from {url}")
            continue
        
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        jobs_on_page = soup.find_all('article')

        # Process each job on the page
        for job in jobs_on_page:
            job_counter += 1
            
            # Extract job details
            try:
                title = job.get('aria-label', 'N/A')
                job_title.append(title)
            except AttributeError:
                job_title.append('N/A')
                
            job_site.append("Seek.com.au")
            
            try:
                desc = job.find('span', {'data-automation': 'jobShortDescription'}).text
                short_description.append(desc)
            except AttributeError:
                short_description.append('N/A')
            
            try:
                location = job.find('a', {'data-automation': 'jobLocation'}).text
                job_location.append(location)
            except AttributeError:
                job_location.append('N/A')
            
            try:
                sub_class = job.find('a', {'data-automation': 'jobSubClassification'}).text
                job_subclassification.append(sub_class)
            except AttributeError:
                job_subclassification.append('N/A')
            
            try:
                classification = job.find('a', {'data-automation': 'jobClassification'}).text
                job_classification.append(classification)
            except AttributeError:
                job_classification.append('N/A')
            
            try:
                company = job.find('a', {'data-automation': 'jobCompany'}).text
                company_name.append(company)
            except AttributeError:
                company_name.append(np.nan)
            
            print(f'Jobs scraped: {job_counter}', end='\r')

---


# end of scrape


In [7]:
print(len(company_name))
print(len(job_title))
print(len(short_description))

220
220
220


In [8]:
seek_data = {'job_title': job_title,
            'company_name': company_name,
            'short_description': short_description,
            'job_location': job_location,
            'job_classification': job_classification,
            'job_subclassification': job_subclassification,
            'job_site' : job_site
             }

In [9]:
seek_data_df = pd.DataFrame(seek_data)
seek_data_df.reset_index(inplace=True)
seek_data_df.rename(columns= {'index':'job_id'}, inplace=True)
seek_data_df.tail(150)

,job_id,job_title,company_name,short_description,job_location,job_classification,job_subclassification,job_site
70,70,A07 Data Analyst,Davidson,Seeking an experienced Data Analyst for an ini...,Brisbane QLD,(Information & Communication Technology),Other,Seek.com.au
71,71,Data Analyst,Hapana,Collaborate with a diverse team to develop the...,Sydney NSW,(Information & Communication Technology),Database Development & Administration,Seek.com.au
72,72,Investment Data Analyst,CBUS Super,Come be a part of team that focuses on deliver...,Melbourne VIC,(Banking & Financial Services),Analysis & Reporting,Seek.com.au
73,73,Business/Data Analyst - Telematics Fleet Manag...,IntelliTrac,An exciting opportunity for a business/data an...,Bundoora,"(Manufacturing, Transport & Logistics)",Analysis & Reporting,Seek.com.au
74,74,Online Data Analyst - Australia,TELUS International AI INC,TELUS International AI-Data Solutions partners...,Brisbane QLD,(Information & Communication Technology),Other,Seek.com.au
...,...,...,...,...,...,...,...,...
215,215,Senior Data Engineer,Domain Group,We're looking for an experienced Senior Data E...,Pyrmont,(Information & Communication Technology),Other,Seek.com.au
216,216,Senior Database Administrator / Data Engineer,365 Assistance,Drive innovation at 365 Assistance! As a Senio...,Sydney NSW,(Information & Communication Technology),Database Development & Administration,Seek.com.au
217,217,Software / Data Engineer,TeraHelix Ltd,Build cutting-edge data solutions in finance a...,Brisbane QLD,(Information & Communication Technology),Engineering - Software,Seek.com.au
218,218,Data Engineer,National Heavy Vehicle Regulator,Data Engineer will contribute to improving reg...,Brisbane QLD,(Information & Communication Technology),Database Development & Administration,Seek.com.au


In [10]:
print(seek_data_df.dtypes)

job_id                    int64
job_title                object
company_name             object
short_description        object
job_location             object
job_classification       object
job_subclassification    object
job_site                 object
dtype: object


In [38]:
#Adding the state based on job location.

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="myGeocoder")

# Initialize the geocoder
geolocator = Nominatim(user_agent="myGeocoder")

# Function to get state from location with retry mechanism
def get_state(location, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(location)
            if location:
                address = geolocator.reverse((location.latitude, location.longitude), exactly_one=True)
                if address:
                    address_info = address.raw.get('address', {})
                    return address_info.get('state', 'State not found')
            return 'Location not found'
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(2)  # Wait before retrying
            else:
                return f'Error: {e}'


# Apply the function to the DataFrame to add state
seek_data_df['State'] = seek_data_df['job_location'].apply(get_state)

In [39]:
print(seek_data_df.dtypes)

job_id                    int64
job_title                object
company_name             object
short_description        object
job_location             object
job_classification       object
job_subclassification    object
job_site                 object
State                    object
dtype: object


In [16]:
seek_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   job_id                 220 non-null    int64 
 1   job_title              220 non-null    object
 2   company_name           219 non-null    object
 3   short_description      220 non-null    object
 4   job_location           220 non-null    object
 5   job_classification     220 non-null    object
 6   job_subclassification  220 non-null    object
 7   job_site               220 non-null    object
dtypes: int64(1), object(7)
memory usage: 13.9+ KB


In [18]:
from geopy.geocoders import Nominatim

# Initialize the geocoder
geolocator = Nominatim(user_agent="myGeocoder")


# Function to get state from location
def get_state(location):
    try:
        location = geolocator.geocode(location)
        if location:
            address = geolocator.reverse((location.latitude, location.longitude), exactly_one=True)
            if address:
                address_info = address.raw.get('address', {})
                return address_info.get('state', location)
        return 'Location not found'
    except Exception as e:
        return 'Error occurred'

# Apply the function to extract states
seek_data_df['State'] = seek_data_df['job_location'].apply(get_state)

# Count the occurrences of each state
#state_counts = df['State'].value_counts()

# Print the result
#print(state_counts)


In [19]:
# Function to remove specific characters like parentheses
def remove_parentheses(text):
    if isinstance(text, str):  # Ensure the input is a string
        text = text.replace('(', '')  # Remove opening parenthesis
        text = text.replace(')', '')  # Remove closing parenthesis
        return text
    return text

# Apply the function to each element in the DataFrame
seek_data_df_cleaned = seek_data_df.apply(lambda col: col.map(remove_parentheses) if col.dtype == 'object' else col)

# Display the cleaned DataFrame
seek_data_df_cleaned

,job_id,job_title,company_name,short_description,job_location,job_classification,job_subclassification,job_site,State
0,0,Data Analyst - 6 month contract - Hybrid,SEEK Limited,Work within the Growth Operations and Analyst ...,Cremorne,Consulting & Strategy,Analysts,Seek.com.au,New South Wales
1,1,Data Analyst,Martin Brower,"If you have a strong analytical mindset, excel...",Wetherill Park,Information & Communication Technology,Business/Systems Analysts,Seek.com.au,New South Wales
2,2,Data Analyst - Policy,"Talent – Specialists in tech, transformation &...","Lead and support funding policy, complex model...",Melbourne VIC,Government & Defence,"Policy, Planning & Regulation",Seek.com.au,Victoria
3,3,National Data Analyst,Capital Transport VIC,We are seeking a National Data Analyst to join...,Clayton,"Manufacturing, Transport & Logistics",Analysis & Reporting,Seek.com.au,Georgia
4,4,Junior Data Analyst,Randstad Digital,We are seeking a junior Data analyst on an ini...,Brisbane QLD,Information & Communication Technology,Other,Seek.com.au,Queensland
...,...,...,...,...,...,...,...,...,...
215,215,Senior Data Engineer,Domain Group,We're looking for an experienced Senior Data E...,Pyrmont,Information & Communication Technology,Other,Seek.com.au,New South Wales
216,216,Senior Database Administrator / Data Engineer,365 Assistance,Drive innovation at 365 Assistance! As a Senio...,Sydney NSW,Information & Communication Technology,Database Development & Administration,Seek.com.au,New South Wales
217,217,Software / Data Engineer,TeraHelix Ltd,Build cutting-edge data solutions in finance a...,Brisbane QLD,Information & Communication Technology,Engineering - Software,Seek.com.au,Queensland
218,218,Data Engineer,National Heavy Vehicle Regulator,Data Engineer will contribute to improving reg...,Brisbane QLD,Information & Communication Technology,Database Development & Administration,Seek.com.au,Queensland


In [29]:
#Saving the data to the flat file (csv)  for displaying it out on the map based on the state and number of job on that state

state_counts = seek_data_df_cleaned['State'].value_counts()

state_counts.to_csv('state_job_count.csv')

---
## Create dataframes for tables
---


In [30]:
# Create job_id dataframe

job_id_df = seek_data_df_cleaned[['job_id', 'job_classification', 'job_location']].copy()

# rename columns to match schema
job_id_df.rename(columns= {'job_classification':'job_field'}, inplace=True)

job_id_df

,job_id,job_field,job_location
0,0,Consulting & Strategy,Cremorne
1,1,Information & Communication Technology,Wetherill Park
2,2,Government & Defence,Melbourne VIC
3,3,"Manufacturing, Transport & Logistics",Clayton
4,4,Information & Communication Technology,Brisbane QLD
...,...,...,...
215,215,Information & Communication Technology,Pyrmont
216,216,Information & Communication Technology,Sydney NSW
217,217,Information & Communication Technology,Brisbane QLD
218,218,Information & Communication Technology,Brisbane QLD


### Count words to plot - All Employment Fields


In [31]:
# Word Cloud Data

full_count = (seek_data_df['short_description'].str.split(expand=True)
              .stack())
full_count_df = pd.DataFrame(full_count)
full_count_df = full_count_df[full_count_df[0].str.len() > 4]
full_count_df

0
0   1          within
    3          Growth
    4      Operations
    6         Analyst
    7           Team,
...               ...
219 9     experienced
    11       Engineer
    14          their
    15      expanding
    16  organisation.

[2093 rows x 1 columns]

In [32]:
# count, tally and describe words in short_description

seek_count = (seek_data_df['short_description'].str.split(expand=True)
              .stack()
              .value_counts()
              .rename_axis('word')
              .reset_index(name='count'))

In [33]:
seek_count = seek_count[seek_count['word'].str.len() > 4]
seek_count

,word,count
10,Engineer,47
14,Analyst,34
24,seeking,22
25,opportunity,19
26,contract,19
...,...,...
1158,extracting,1
1160,Chain,1
1161,Planning,1
1162,Melbourne.,1


In [34]:
seek_count.head(45)

,word,count
10,Engineer,47
14,Analyst,34
24,seeking,22
25,opportunity,19
26,contract,19
27,business,18
30,drive,16
33,within,15
38,Senior,13
39,support,13


### Count words to plot - Individual fields (Data-Analyst, Data-Visualisation)


In [20]:
# Create DF for each job field

data_analyst_df = seek_data_df_cleaned[seek_data_df_cleaned['job_classification']=='Banking & Financial Services']
data_engineer_df = seek_data_df_cleaned[seek_data_df_cleaned['job_classification']=='Information & Communication Technology']

In [21]:
# create word count DF for each job field
# word cloud
# count, tally and describe words in short_description
import regex
from collections import Counter

# construction

data_analyst_count = (data_analyst_df['short_description'].str.split(expand=True)
              .stack()
              .value_counts()
              .rename_axis('word')
              .reset_index(name='count'))

# engineering

data_engineer_count = (data_engineer_df['short_description'].str.split(expand=True)
              .stack()
              .value_counts()
              .rename_axis('word')
              .reset_index(name='count'))

In [22]:
# word cloud


# analyst

# Flatten the DataFrame, split into words, and remove non-word characters
data_analyst_set = data_analyst_df.stack().str.split(r'\W+', expand=False).explode().tolist()

# Convert to DataFrame
data_analyst_count_df = pd.DataFrame(data_analyst_set, columns=['Word'])
# Filter words with length greater than 4 characters
data_analyst_count_df = data_analyst_count_df[data_analyst_count_df['Word'].str.len() > 4]
# Count frequencies of each word
data_analyst_count_df = data_analyst_count_df['Word'].value_counts().reset_index()
data_analyst_count_df.columns = ['Word', 'Count']

# engineer

# Flatten the DataFrame, split into words, and remove non-word characters
data_engineer_set = data_engineer_df.stack().str.split(r'\W+', expand=False).explode().tolist()

# Convert to DataFrame
data_engineer_count_df = pd.DataFrame(data_engineer_set, columns=['Word'])
# Filter words with length greater than 4 characters
data_engineer_count_df = data_engineer_count_df[data_engineer_count_df['Word'].str.len() > 4]
# Count frequencies of each word
data_engineer_count_df = data_engineer_count_df['Word'].value_counts().reset_index()
data_engineer_count_df.columns = ['Word', 'Count']

# Display the result
data_engineer_count_df.head(40)

,Word,Count
0,Technology,189
1,Information,184
2,Communication,183
3,Engineer,146
4,Analyst,102
5,Business,66
6,Analysts,59
7,Systems,59
8,Senior,54
9,Engineering,54


In [23]:
# Convert the column to string type before applying string operations
data_analyst_count['word'] = data_analyst_count['word'].astype(str)
data_engineer_count['word'] = data_engineer_count['word'].astype(str)

In [24]:
data_analyst_count = data_analyst_count[data_analyst_count['word'].str.len() > 4]
data_engineer_count = data_engineer_count[data_engineer_count['word'].str.len() > 4]

In [25]:
print("Data Engineer Count:")
print(data_engineer_count)

Data Engineer Count:
             word  count
9        Engineer     46
18        Analyst     21
23        seeking     18
26         Senior     14
29       contract     13
...           ...    ...
1013  significant      1
1014      impact?      1
1016       Budget      1
1017       effort      1
1018    decision.      1

[807 rows x 2 columns]


### Count locations to plot

---


In [ ]:
# seek_data_df
# data_engineer_df

In [ ]:
# create df for location counts 
# heatmap plot

all_fields_locations = seek_data_df_cleaned['job_location'].value_counts()
data_analyst_locations = data_analyst_df['job_location'].value_counts()
data_engineer_locations = data_engineer_df['job_location'].value_counts()


all_fields_locations_df = pd.DataFrame(all_fields_locations)
data_analyst_locations_df = pd.DataFrame(data_analyst_locations)
data_engineer_locations_df = pd.DataFrame(data_engineer_locations)

data_engineer_locations

job_location
Sydney NSW        50
Brisbane QLD      24
Canberra ACT      23
Melbourne VIC     22
Adelaide SA        8
Perth WA           6
Belmont            4
Alexandria         3
Wetherill Park     2
Geelong            2
Gosford            2
Hobart TAS         2
Booragoon          2
Collingwood        2
Artarmon           2
Surry Hills        2
Woodville          2
Maroochydore       2
Griffith           1
Osborne            1
Broadbeach         1
Pyrmont            1
Mascot             1
Burwood            1
Norwest            1
Capital Hill       1
Richmond           1
Parramatta         1
Cremorne           1
Annerley           1
Eagle Farm         1
Docklands          1
Henderson          1
Laverton North     1
Dandenong          1
Enoggera           1
Box Hill           1
Mulgrave           1
Gold Coast QLD     1
Macquarie Park     1
Smithfield         1
Balaclava          1
Kent Town          1
Name: count, dtype: int64

In [ ]:
# reset index for tables
all_fields_locations_df.reset_index(inplace=True)
data_analyst_locations_df.reset_index(inplace=True)
data_engineer_locations_df.reset_index(inplace=True)


In [ ]:
# rename columns to be more friendly
all_fields_locations_df.rename(columns= {'index':'job_location', 'job_location':'count'}, inplace=True)
data_analyst_locations_df.rename(columns= {'index':'job_location', 'job_location':'count'}, inplace=True)
data_engineer_locations_df.rename(columns= {'index':'job_location', 'job_location':'count'}, inplace=True)

data_engineer_locations_df

,count,count
0,Sydney NSW,50
1,Brisbane QLD,24
2,Canberra ACT,23
3,Melbourne VIC,22
4,Adelaide SA,8
5,Perth WA,6
6,Belmont,4
7,Alexandria,3
8,Wetherill Park,2
9,Geelong,2


---


### Connect to postgres and load data to db

---


In [ ]:
# MOVE TO TOP BEFORE SUBMITTING
# from api_keys import postgres_p
from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2

In [ ]:
protocol = 'postgresql'
username = 'postgres'
password = "Jane2801kei!"

host = 'localhost'

port = 5432

database_name = 'Project_3_1'

rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

insp.get_table_names()

['job_id', 'data_analyst_scrapes', 'data_engineer_scrapes']

In [ ]:
data_engineer_df

,job_id,job_title,company_name,short_description,job_location,job_classification,job_subclassification,job_site
1,1,Data Analyst,Martin Brower,"If you have a strong analytical mindset, excel...",Wetherill Park,Information & Communication Technology,Business/Systems Analysts,Seek.com.au
3,3,Junior Data Analyst,Kerry Logistics Cold Chain Australia Pty Ltd,Global Company - Laverton North,Laverton North,Information & Communication Technology,Business/Systems Analysts,Seek.com.au
4,4,Data Analyst | Digital Media Products,ABC,Use audience data to generate insights that ad...,Sydney NSW,Information & Communication Technology,Business/Systems Analysts,Seek.com.au
5,5,Data Analyst | Digital Media Products,ABC,Use audience data to generate insights that ad...,Melbourne VIC,Information & Communication Technology,Business/Systems Analysts,Seek.com.au
6,6,Data Analyst | Digital Media Products,ABC,Use audience data to generate insights that ad...,Brisbane QLD,Information & Communication Technology,Business/Systems Analysts,Seek.com.au
...,...,...,...,...,...,...,...,...
214,214,Data Engineer,National Heavy Vehicle Regulator,Data Engineer will contribute to improving reg...,Brisbane QLD,Information & Communication Technology,Database Development & Administration,Seek.com.au
215,215,Data Engineer - OT experience,Akkodis,"Having a global footprint with 50,000+ thinker...",Perth WA,Information & Communication Technology,Engineering - Software,Seek.com.au
216,216,"Support Data Engineer, Snowflake",MUFG Retirement Solutions,5+ yrs of exp as a Data/Analytics Engineer wit...,Melbourne VIC,Information & Communication Technology,Other,Seek.com.au
217,217,Data and Analytics Developer,Qube Logistics,Qube is seeking a skilled Data and Analytics D...,Sydney NSW,Information & Communication Technology,Database Development & Administration,Seek.com.au


In [ ]:
# load to database

# Job_ID Table

job_id_df.to_sql(name='job_id', con=engine, if_exists='append', index=False)

## Scraped job Tables

data_analyst_df.to_sql(name='data_analyst_scrapes', con=engine, if_exists='append', index=False)
data_engineer_df.to_sql(name='data_engineer_scrapes', con=engine, if_exists='append', index=False)

185

In [ ]:
pd.read_sql_query('select * from data_analyst_scrapes', con=engine).head(15)

,job_id,job_title,company_name,short_description,job_location,job_classification,job_subclassification,job_site
0,14,Data Analyst,CBHS Health Fund Limited,Our members are at the heart of everything we ...,Parramatta,Banking & Financial Services,Analysis & Reporting,Seek.com.au
1,15,Data Analyst,"Talent – Specialists in tech, transformation &...","Our client is a leading financial institution,...",Sydney NSW,Banking & Financial Services,Analysis & Reporting,Seek.com.au
2,21,Data Analyst,McMillan Shakespeare,Bring your data analysis skills to conduct ana...,Melbourne VIC,Banking & Financial Services,Analysis & Reporting,Seek.com.au
3,42,Investment Data Analyst,CBUS Super,Come be a part of team that focuses on deliver...,Melbourne VIC,Banking & Financial Services,Analysis & Reporting,Seek.com.au
4,44,Data Analyst,CBHS Health Fund Limited,Our members are at the heart of everything we ...,Parramatta,Banking & Financial Services,Analysis & Reporting,Seek.com.au


In [ ]:
# ## Word counts

data_analyst_count.to_sql(name='data_analyst_word_count', con=engine, if_exists='append', index=False)
data_engineer_count.to_sql(name='data_engineer_word_count', con=engine, if_exists='append', index=False)

844

In [ ]:
pd.read_sql_query('select * from data_engineer_word_count', con=engine).head(15)

,word,count
0,Engineer,51
1,seeking,24
2,Analyst,19
3,critical,14
4,contract,14
5,Senior,14
6,opportunity,13
7,Azure,12
8,looking,12
9,within,12


In [ ]:
# ## Location Tables

all_fields_locations_df.to_sql(name='allfields_location_count', con=engine, if_exists='append', index=False)

DuplicateColumnError: A column with name 'count' is already present in table 'allfields_location_count'.